# Reading data

In [ ]:
import pickle

# Define the file path
file_path_train = "measurements_by_patient_train.pkl"
file_path_test = "measurements_by_patient_test.pkl"

with open(file_path_train, 'rb') as file:
    train = pickle.load(file)

with open(file_path_test, 'rb') as file:
    test = pickle.load(file)

# Generating windows

This section create the windows for the new two horizons (90 and 120 minutes)

In [ ]:
import numpy as np

def get_windows_one_step_walk_forward(data, lookback_samples, pred_samples):
  """

    :param data:
    :param lookback_samples: Number of samples used to predict
    :param pred_samples: Prediction window (normally 30 or 60 minutes)
    :param backup: Save the progress each 100 patients
    :return: Get windows without missing values from one step sliding windows
    """
  x_per_patient = []
  y_per_patient = []

  for data_patient in data:

    # Creating the one step sliding window
    x = np.lib.stride_tricks.sliding_window_view(data_patient[:-pred_samples], lookback_samples)
    y = np.lib.stride_tricks.sliding_window_view(data_patient[lookback_samples:], pred_samples)

    # Removing rows with missing values
    nan_rows_x = np.isnan(x).any(axis=1)
    nan_rows_y = np.isnan(y).any(axis=1)
    x = x[~(nan_rows_x | nan_rows_y)]
    y = y[~(nan_rows_x | nan_rows_y)]

    x_per_patient.append(x)
    y_per_patient.append(y)

  x_result = np.concatenate(x_per_patient)
  y_result = np.concatenate(y_per_patient)

  return x_result, y_result



history_length = 8
horizons = [6, 8] #90 and 120 min

for hor in horizons:
    x_train, y_train = get_windows_one_step_walk_forward(train, history_length, hor)
    x_test, y_test = get_windows_one_step_walk_forward(test, history_length, hor)
    np.save('windows/x_train_windows_horizon_{0}.npy'.format(hor), x_train)     # NOTE: Replace with your desired filepath
    np.save('windows/y_train_windows_horizon_{0}.npy'.format(hor), y_train)
    np.save('windows/x_test_windows_horizon_{0}.npy'.format(hor), x_test)
    np.save('windows/y_test_windows_horizon_{0}.npy'.format(hor), y_test)
    
    

# Auxiliar Functions

## Windows

In [ ]:
import numpy as np 

def read_windows(horizon):
  file_x_train = f"windows/x_train_windows_horizon_{horizon}.npy"
  file_y_train = f"windows/y_train_windows_horizon_{horizon}.npy"
  file_x_test  = f"windows/x_test_windows_horizon_{horizon}.npy"
  file_y_test  = f"windows/y_test_windows_horizon_{horizon}.npy"
  
  print(f"Reading training files {file_x_train} and {file_y_train}...")
  print(f"Reading test files {file_x_test} and {file_y_test}...")
  
  x_train = np.load(file_x_train)
  y_train = np.load(file_y_train)
  x_test = np.load(file_x_test)
  y_test = np.load(file_y_test)

  # Using only the horizon measurement as a label
  y_train = y_train[:, -1]
  y_train = y_train.reshape((y_train.shape[0], 1))
  y_test = y_test[:, -1]
  y_test = y_test.reshape((y_test.shape[0], 1))

  return x_train, y_train, x_test, y_test



## LSTM and linear model architectures

In [ ]:
from keras.models import Model
from keras.layers import Dense, LSTM, GRU, Lambda, dot, concatenate, Activation, Input

# LSTM
class LSTMModel:
    def __init__(self, input_shape, nb_output_units, nb_hidden_units=128):
        self.input_shape = input_shape
        self.nb_output_units = nb_output_units
        self.nb_hidden_units = nb_hidden_units
        
    def __repr__(self):
        return 'LSTM_{0}_units_{1}_layers_dropout={2}_{3}'.format(self.nb_hidden_units, self.nb_layers, self.dropout, self.recurrent_dropout)
    
    def build(self):
        # input
        i = Input(shape=self.input_shape)

        # add LSTM layer
        x = LSTM(self.nb_hidden_units)(i)

        x = Dense(self.nb_output_units, activation=None)(x)

        return Model(inputs=[i], outputs=[x])
    
# Linear
class LinearModel:
    def __init__(self, input_shape, nb_output_units):
        self.input_shape = input_shape
        self.nb_output_units = nb_output_units

    def __repr__(self):
        return 'Linear'

    def build(self):
        i = Input(shape=self.input_shape)
        x = Dense(self.nb_output_units, activation=None)(i)

        return Model(inputs=[i], outputs=[x])


## Model Functions

In [ ]:
from keras.callbacks import ModelCheckpoint, EarlyStopping
import keras.backend as K

def RMSE(output, target):
    output = K.cast(output, 'float32')
    target = K.cast(target, 'float32')

    return K.sqrt(K.mean((output - target) ** 2))

def build_model(model, weights=''):
  
  # build & compile model
  m = model.build()

  m.compile(loss=RMSE,
            optimizer='adam',
            metrics=[RMSE])
  if weights:
    print(f"Weights: {weights}")
    m.load_weights(weights)
  return m

def callbacks(filepath, early_stopping_patience):
  callbacks = []
  callbacks.append(ModelCheckpoint(filepath=filepath,
                                  monitor='RMSE',
                                  save_best_only=True,
                                  save_weights_only=True))
  callbacks.append(EarlyStopping(monitor='RMSE', patience=early_stopping_patience))
  return callbacks

In [ ]:

import numpy as np

def prepare_model_LSTM(history_length, nb_hidden_units=128, weights=''):
  model = LSTMModel(input_shape=(history_length, 1), nb_output_units=1, nb_hidden_units=nb_hidden_units)
  return build_model(model, weights)

def prepare_model_linear(history_length, weights=''):
  model = LinearModel(input_shape=(history_length,), nb_output_units=1)
  return build_model(model, weights)
 
 
 
 
def train(x_train, y_train, model, horizon, save_filepath = "", early_stopping_patience=50):

  history_length = 8
  
  if repr(model)[0:4] == 'LSTM':
    # Reshaping the data (needed for LSTM model)
    x_train = np.reshape(x_train, (x_train.shape[0], history_length, 1))

  # Train
  hist = model.fit(x_train, y_train,
                batch_size=32,
                epochs=500,
                #shuffle=True,
                callbacks=callbacks(save_filepath + repr(model) + str(horizon), 
                                    early_stopping_patience)
                )


  return hist

# Training

In this sections the models are trained for the new horizons

In [ ]:
history_length = 8 # 2h

x_train_6, y_train_6, x_test, y_test = read_windows(horizon=6)
x_train_8, y_train_8, x_test, y_test = read_windows(horizon=8)

LSTM_model = prepare_model_LSTM(history_length)
linear_model = prepare_model_linear(history_length)

# NOTE: Be sure that the save_filepath is a real path. If not, the callback
#won't save your model and you will lose your progress

print("########################################################################")
hist = train(x_train_6, y_train_6, LSTM_model, horizon=6, save_filepath="")
print("########################################################################")
print()

print("########################################################################")
hist = train(x_train_8, y_train_8, LSTM_model, horizon=8, save_filepath="")
print("########################################################################")
print()

print("########################################################################")
hist = train(x_train_6, y_train_6, linear_model, horizon=6, save_filepath="")
print("########################################################################")
print()

print("########################################################################")
hist = train(x_train_8, y_train_8, linear_model, horizon=8, save_filepath="")
print("########################################################################")
print()

